<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Andrés Oñate
- Nombre de alumno 2: Javier Zapata


Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/andresignacio-o/MDS7202.git)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("sales.csv")

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [ ]:
#valor max y min en date
min_date = df['date'].min()
max_date = df['date'].max()

print("Fecha mínima:", min_date)
print("Fecha máxima:", max_date)

Fecha mínima: 28/02/13
Fecha máxima: 31/12/18


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [ ]:
"""
ME GENERA DUDA REALMENTE LO QUE HAY QUE HACER PQ AL PEDIR SEED ENTIENDO QUE ES PARA
TEMAS RANDOM, PERO SIGUIENDO BUENAS PRACTICAS DEBERIAMOS HACER UNA SEPARACION TEMPORAL
DONDE EL TRAIN SEA EL 70% INICIAL, VALIDACION EL SIGUIENTE 20% Y TEST EL 10% FINAL.

ABAJO LO HARE CON SEED

"""

from sklearn import set_config
set_config(transform_output="pandas")

# Inserte su código acá

#separar en train, validación y test con fechas min y max
df = df.sort_values('date')  # ordenar por fecha ascendente

# calcular índices de corte
n = len(df)
train_end = int(0.7 * n)
val_end = int(0.9 * n)  # 70 + 20 = 90%

# separar
train = df.iloc[:train_end]
val = df.iloc[train_end:val_end]
test = df.iloc[val_end:]

print(f"Train: {len(train)} filas, Val: {len(val)} filas, Test: {len(test)} filas")

Train: 5219 filas, Val: 1491 filas, Test: 746 filas


In [ ]:
train.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
1182,1199,28/02/13,Patra,38.24444,21.73444,166301,shop_6,orange-power,plastic,1.5lt,2.13,17286
1179,1196,28/02/13,Patra,38.24444,21.73444,166301,shop_6,adult-cola,glass,500ml,1.31,14484
1180,1197,28/02/13,Patra,38.24444,21.73444,166301,shop_6,adult-cola,plastic,1.5lt,3.17,10108
1181,1198,28/02/13,Patra,38.24444,21.73444,166301,shop_6,adult-cola,can,330ml,0.54,34234
1183,1200,28/02/13,Patra,38.24444,21.73444,166301,shop_6,orange-power,can,330ml,0.46,35761


In [ ]:
seed = 1234

# --- 1) Splits 70/20/10 (aleatorio con semilla) ---
from sklearn.model_selection import train_test_split
train, temp = train_test_split(df, test_size=0.30, random_state=seed, shuffle=True)
val, test = train_test_split(temp, test_size=1/3, random_state=seed, shuffle=True)

# --- 2) FunctionTransformer: extraer partes de fecha como 'category' ---
from sklearn.preprocessing import FunctionTransformer

DATE_FMT = "%d-%m-%y"   # ajusta
def extract_date_features(X):
    X = X.copy()
    if not np.issubdtype(X['date'].dtype, np.datetime64):
        X['date'] = pd.to_datetime(X['date'], format=DATE_FMT, errors='coerce')
    X['day']   = X['date'].dt.day.astype('category')
    X['month'] = X['date'].dt.month.astype('category')
    X['year']  = X['date'].dt.year.astype('category')
    return X.drop(columns=['date'])

date_transformer = FunctionTransformer(extract_date_features)

# --- 3) ColumnTransformer con detección automática y set_output='pandas' ---
import numpy as np
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

numeric_pipe = Pipeline([
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline([
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, selector(dtype_include=np.number)),     # todas las numéricas
        ("cat", categorical_pipe, selector(dtype_include="category")) # todas las categóricas (incluye day/month/year)
    ],
    remainder="drop",
    verbose_feature_names_out=False
).set_output(transform="pandas")  # <-- solicitado

# 4) Pipeline con DummyRegressor (baseline)
from sklearn.dummy import DummyRegressor

baseline_pipe = Pipeline(steps=[
    ("date_features", date_transformer),
    ("prep", preprocessor),
    ("reg", DummyRegressor(strategy="mean"))
])



# ===========================
# 5) Entrenar y evaluar en VALIDACIÓN (MAE)
# ===========================

TARGET_COL = "quantity"

from sklearn.metrics import mean_absolute_error

# Split X/y para cada set
X_train, y_train = train.drop(columns=[TARGET_COL]), train[TARGET_COL]
X_val,   y_val   = val.drop(columns=[TARGET_COL]),   val[TARGET_COL]
X_test,  y_test  = test.drop(columns=[TARGET_COL]),  test[TARGET_COL]

# Entrenar baseline
baseline_pipe.fit(X_train, y_train)
val_pred_dummy = baseline_pipe.predict(X_val)
mae_dummy = mean_absolute_error(y_val, val_pred_dummy)
print(f"[DummyRegressor] MAE validación: {mae_dummy:,.4f}")


# ===========================
# 6) Pipeline con XGBRegressor (default) y comparación
# ===========================
from xgboost import XGBRegressor

xgb_pipe = Pipeline(steps=[
    ("date_features", date_transformer),
    ("prep", preprocessor),
    ("reg", XGBRegressor(
        random_state=seed,    # reproducibilidad
        # parámetros por default (explícitos para claridad; puedes omitirlos si quieres)
        n_estimators=100,
        learning_rate=0.3,
        max_depth=6,
        subsample=1.0,
        colsample_bytree=1.0,
        reg_lambda=1.0,
        tree_method="hist"
    ))
])

xgb_pipe.fit(X_train, y_train)
val_pred_xgb = xgb_pipe.predict(X_val)
mae_xgb = mean_absolute_error(y_val, val_pred_xgb)
print(f"[XGBRegressor] MAE validación: {mae_xgb:,.4f}")

if mae_xgb < mae_dummy:
    print(" XGBRegressor mejora el MAE respecto al DummyRegressor.")
elif mae_xgb > mae_dummy:
    print(" XGBRegressor tiene MAE peor que el DummyRegressor (revisar features/overfitting).")
else:
    print("⏸ Ambos modelos empatan en MAE.")


# ===========================
# 7) Guardar ambos modelos en .pkl
# ===========================
import joblib
joblib.dump(baseline_pipe, "baseline_dummy.pkl")
joblib.dump(xgb_pipe, "model_xgb.pkl")
print("Modelos guardados: baseline_dummy.pkl y model_xgb.pkl")

[DummyRegressor] MAE validación: 13,055.2009
[XGBRegressor] MAE validación: 7,087.9985
 XGBRegressor mejora el MAE respecto al DummyRegressor.
Modelos guardados: baseline_dummy.pkl y model_xgb.pkl


## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [ ]:
# Inserte su código acá

# === P1 ===

# === Config mínimas (ajusta si tus nombres son distintos) ===
TARGET_COL = "quantity"
PRICE_COL  = "price"

# === 1) Obtener nombres de columnas antes del modelo (tras prep) ===
from sklearn.pipeline import Pipeline

X_train, y_train = train.drop(columns=[TARGET_COL]), train[TARGET_COL]
X_val,   y_val   = val.drop(columns=[TARGET_COL]),   val[TARGET_COL]

# Reutiliza tus OBJETOS existentes: date_transformer y preprocessor
proc_only = Pipeline([
    ("date_features", date_transformer),
    ("prep", preprocessor)
])
proc_only.fit(X_train, y_train)  # y no se usa, pero no molesta
feat_names = proc_only.named_steps["prep"].get_feature_names_out()

# === 2) Construir constraints por NOMBRE: precio con -1, resto 0 ===
# Intento primero nombre exacto; si no está, busco alguna variante (p.ej. sufijo)
if PRICE_COL in feat_names:
    price_col_name = PRICE_COL
else:
    candidates = [c for c in feat_names if c.endswith(PRICE_COL)]
    if not candidates:
        raise ValueError(
            f"No encontré la columna de precio en las features transformadas. "
            f"Revisa feat_names (p.ej., print(list(feat_names)) )"
        )
    price_col_name = candidates[0]

constraints = {name: 0 for name in feat_names}
constraints[price_col_name] = -1  # relación monótona NEGATIVA: ↑precio → ↓cantidad

# === 3) Inyectar constraints en TU pipeline existente y reentrenar ===
# (no reescribo xgb_pipe; solo set_params sobre el paso 'reg')
xgb_pipe.set_params(reg__monotone_constraints=constraints)

xgb_pipe.fit(X_train, y_train)

# === P2 ===
from sklearn.metrics import mean_absolute_error
val_pred_xgb_mono = xgb_pipe.predict(X_val)
mae_xgb_mono = mean_absolute_error(y_val, val_pred_xgb_mono)
print(f"[XGBRegressor + monotone(price→-)] MAE validación: {mae_xgb_mono:,.4f}")



# === P3 ===
#En texto abajo

# === P4 ===

# Guarda el pipeline entrenado con monotonicidad
joblib.dump(xgb_pipe, "xgb_monotone.pkl")



[XGBRegressor + monotone(price→-)] MAE validación: 7,201.3154


['xgb_monotone.pkl']

Tras la incorporación de la restricción monótona negativa entre el precio y la cantidad demandada, el error absoluto medio (MAE) del modelo aumentó levemente, pasando de 7.015,6 a 7.122,0 unidades.
Desde una perspectiva académica, este resultado sugiere que imponer una restricción de monotonicidad introduce una forma de regularización estructural en el modelo: se fuerza al estimador a respetar una relación teóricamente esperada (mayor precio → menor demanda), reduciendo su flexibilidad para capturar patrones idiosincráticos o ruidos presentes en los datos.
En este caso, la ligera degradación del desempeño en validación indica que los datos reales no siguen estrictamente una relación decreciente entre precio y cantidad. Es plausible que existan efectos de confusión (por ejemplo, descuentos por volumen, promociones, diferencias entre productos o estacionalidad) que generen observaciones donde el precio no explica de manera monótona la demanda.
Por lo tanto, si bien el modelo con restricción refleja una hipótesis económica consistente y podría mejorar la interpretabilidad y generalización futura, empíricamente su ajuste en este conjunto se ve levemente afectado.
En la práctica la relación no es perfectamente monótona en los datos disponibles.

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
# ===========================
# Optuna: TPE sobre XGB + OHE.min_frequency
# ===========================
import numpy as np
import optuna
from optuna.samplers import TPESampler

from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Split X/y (reutilizamos tus dataframes)
X_train, y_train = train.drop(columns=[TARGET_COL]), train[TARGET_COL]
X_val,   y_val   = val.drop(columns=[TARGET_COL]),   val[TARGET_COL]

# Para reproducibilidad adicional
np.random.seed(seed)
optuna.logging.set_verbosity(optuna.logging.WARNING)
sampler = TPESampler(seed=seed)  # <-- TPE con seed

def build_pipeline(ohe_min_freq: float, xgb_params: dict) -> Pipeline:
    """
    Clona tu preprocesamiento (manteniendo .set_output('pandas'))
    y construye el pipeline con XGBRegressor.
    """
    proc = clone(preprocessor)             # clonar tu ColumnTransformer
    dt   = clone(date_transformer)         # clonar tu FunctionTransformer

    pipe = Pipeline(steps=[
        ("date_features", dt),
        ("prep", proc),
        ("reg", XGBRegressor(**xgb_params))
    ])

    # Ajustar el hiperparámetro del OneHotEncoder dentro de tu rama categórica
    # Tu preprocessor tiene ("cat" -> "ohe")
    pipe.set_params(prep__cat__ohe__min_frequency=ohe_min_freq)

    return pipe

def objective(trial: optuna.trial.Trial) -> float:
    # ---- Espacio de búsqueda requerido ----
    params = dict(
        learning_rate   = trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        n_estimators    = trial.suggest_int("n_estimators", 50, 1000),
        max_depth       = trial.suggest_int("max_depth", 3, 10),
        max_leaves      = trial.suggest_int("max_leaves", 0, 100),
        min_child_weight= trial.suggest_int("min_child_weight", 1, 5),
        reg_alpha       = trial.suggest_float("reg_alpha", 0.0, 1.0),
        reg_lambda      = trial.suggest_float("reg_lambda", 0.0, 1.0),
        random_state    = seed,       # reproducibilidad
        tree_method     = "hist",     # rápido y estable
        n_jobs          = -1
    )
    ohe_min_freq = trial.suggest_float("ohe_min_frequency", 0.0, 1.0)

    pipe = build_pipeline(ohe_min_freq, params)
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    # Guardar el mejor pipeline entrenado en el study
    # (accesible luego en study.user_attrs["best_pipeline"])
    study = trial.study
    if "best_mae" not in study.user_attrs or mae < study.user_attrs["best_mae"]:
        study.set_user_attr("best_mae", mae)
        study.set_user_attr("best_pipeline", pipe)

    return mae

# ---- Ejecutar la optimización por tiempo (5 minutos = 300s) ----
study = optuna.create_study(direction="minimize", sampler=sampler)
study.optimize(objective, timeout=300)  # <-- 5 minutos

# ---- Reporte solicitado ----
n_trials = len(study.trials)
best_mae = study.best_value
best_params = study.best_params

print(f"Trials ejecutados: {n_trials}")
print(f"Mejor MAE (validación): {best_mae:,.4f}")
print("Mejores hiperparámetros encontrados:")
for k, v in best_params.items():
    print(f"  - {k}: {v}")

# Comparación con tu mejor sección anterior (ajusta si cambian):
prev_default_mae   = 7015.6182   # XGB por defecto (tu output previo)
prev_monotone_mae  = 7122.0000   # XGB con restricción monótona (tu output previo)
print(f"\nΔ vs XGB default: {best_mae - prev_default_mae:,.4f} (negativo = mejora)")
print(f"Δ vs XGB monotono: {best_mae - prev_monotone_mae:,.4f} (negativo = mejora)")


#Guardar modelo
...


Trials ejecutados: 469
Mejor MAE (validación): 6,614.2266
Mejores hiperparámetros encontrados:
  - learning_rate: 0.057783050516976485
  - n_estimators: 944
  - max_depth: 3
  - max_leaves: 56
  - min_child_weight: 3
  - reg_alpha: 0.2323586356498477
  - reg_lambda: 0.9263925527963227
  - ohe_min_frequency: 0.01934909749468694

Δ vs XGB default: -401.3916 (negativo = mejora)
Δ vs XGB monotono: -507.7734 (negativo = mejora)


Pendiente responder:

Optimizar el modelo y reportar el número de trials, el MAE y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
# Inserte su código acá
# ===========================
# Optuna + Pruning: XGB (SIN monotonía) + OHE.min_frequency
# Requiere: train, val, date_transformer, preprocessor, seed, TARGET_COL="quantity"
# ===========================
import numpy as np
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from optuna.integration import XGBoostPruningCallback

from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib, os, shutil

# Silenciar logs ruidosos de Optuna; activamos barra de progreso en .optimize
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Datos
X_train, y_train = train.drop(columns=[TARGET_COL]), train[TARGET_COL]
X_val,   y_val   = val.drop(columns=[TARGET_COL]),   val[TARGET_COL]

# Reproducibilidad
np.random.seed(seed)
sampler = TPESampler(seed=seed)

# Un pruner conservador y popular
pruner = MedianPruner(n_warmup_steps=10)  # espera algunas iteraciones antes de podar

BEST_TMP_PATH = "optuna_prune_best_tmp.pkl"   # sobreescribe cuando haya mejora

def objective(trial: optuna.trial.Trial) -> float:
    params = dict(
        learning_rate     = trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        n_estimators      = trial.suggest_int("n_estimators", 50, 1000),
        max_depth         = trial.suggest_int("max_depth", 3, 10),
        max_leaves        = trial.suggest_int("max_leaves", 0, 100),
        min_child_weight  = trial.suggest_int("min_child_weight", 1, 5),
        reg_alpha         = trial.suggest_float("reg_alpha", 0.0, 1.0),
        reg_lambda        = trial.suggest_float("reg_lambda", 0.0, 1.0),
        random_state      = seed,
        tree_method       = "hist",
        n_jobs            = -1,
        eval_metric       = "mae"   # ✅ aquí va ahora
    )
    ohe_min_freq = trial.suggest_float("ohe_min_frequency", 0.0, 1.0)

    proc = clone(preprocessor)
    dt   = clone(date_transformer)

    pipe = Pipeline(steps=[
        ("date_features", dt),
        ("prep", proc),
        ("reg", XGBRegressor(**params))
    ])
    pipe.set_params(prep__cat__ohe__min_frequency=ohe_min_freq)

    pruning_cb = XGBoostPruningCallback(trial, "validation_0-mae")

    pipe.fit(
        X_train, y_train,
        reg__eval_set=[(X_val, y_val)],
        reg__callbacks=[pruning_cb],
        reg__verbose=False
    )

    y_pred = pipe.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    st = trial.study
    if "best_mae" not in st.user_attrs or mae < st.user_attrs["best_mae"]:
        joblib.dump(pipe, BEST_TMP_PATH)
        st.set_user_attr("best_mae", mae)
        st.set_user_attr("best_model_path", os.path.abspath(BEST_TMP_PATH))
        st.set_user_attr("best_params", {**params, "ohe_min_frequency": ohe_min_freq})

    return mae


# ----- Ejecutar la optimización por tiempo (5 minutos) -----
study = optuna.create_study(direction="minimize", sampler=sampler, pruner=pruner)
study.optimize(objective, timeout=300, show_progress_bar=True)

# ----- Reporte -----
n_trials   = len(study.trials)
best_mae   = study.best_value
best_params= study.best_params
print(f"Trials ejecutados: {n_trials}")
print(f"Mejor MAE (validación): {best_mae:,.4f}")
print("Mejores hiperparámetros encontrados:")
for k, v in best_params.items():
    print(f"  - {k}: {v}")

# (Opcional) Comparación contra tu mejor resultado anterior:
# prev_default_mae   = 7015.6182
# print(f"Δ vs XGB default: {best_mae - prev_default_mae:,.4f} (negativo = mejora)")

# ----- Guardar modelo final en .pkl -----
final_path = "xgb_optuna_pruned_best.pkl"
shutil.copyfile(study.user_attrs["best_model_path"], final_path)
print("Modelo optimizado con pruning guardado en:", final_path)


   0%|          | 00:00/05:00

[W 2025-10-08 14:28:30,695] Trial 0 failed with parameters: {'learning_rate': 0.002415677202529624, 'n_estimators': 641, 'max_depth': 6, 'max_leaves': 79, 'min_child_weight': 4, 'reg_alpha': 0.2725926052826416, 'reg_lambda': 0.2764642551430967, 'ohe_min_frequency': 0.8018721775350193} because of the following error: TypeError("XGBModel.fit() got an unexpected keyword argument 'callbacks'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-1873620991.py", line 62, in objective
    pipe.fit(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last_step_

TypeError: XGBModel.fit() got an unexpected keyword argument 'callbacks'

El pruning (poda temprana) es una técnica de optimización que finaliza de manera anticipada aquellas configuraciones de hiperparámetros (trials) que, según una métrica de validación parcial durante el entrenamiento, no muestran potencial para superar al mejor resultado actual. En lugar de consumir el presupuesto completo (tiempo/iteraciones) en trials poco prometedores, el algoritmo reorienta el cómputo hacia regiones más plausibles del espacio de búsqueda.
En términos prácticos, el pruning acelera la optimización (más trials útiles en el mismo tiempo) y reduce el coste de exploración. Idealmente, esto no empeora la calidad del mejor resultado y, en ocasiones, la mejora al permitir cubrir más configuraciones en el mismo presupuesto temporal. El costo potencial es que decisiones de poda demasiado agresivas pueden descartar trials que mejoraban tarde, por lo que conviene combinarlo con métricas estables y callbacks adecuados.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Exito!
<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>